In [1]:
# !rm -rf tmp_out

In [2]:
!pip install torcheeg torch-scatter torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.4/251.4 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of mne to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of wfdb to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torcheeg.datasets import SEEDIVDataset
from torcheeg import transforms
# from torcheeg.models import EEGNet
import scipy.signal as signal
import random
from torchvision.models import googlenet
import copy

In [4]:
# 1. Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
def BandPassFilter(eeg_data):
    b, a = signal.butter(4, Wn=[1.0, 75.0], btype='bandpass', fs=200)
    return signal.filtfilt(b, a, eeg_data, axis=-1)

In [6]:
def Notch(eeg_data):
    b, a = signal.iirnotch(w0=50.0, Q=30.0, fs=200)
    return signal.filtfilt(b, a, eeg_data, axis=-1)

In [7]:
# 2. Define Preprocessing
t_transform = transforms.Compose([
    transforms.Lambda(BandPassFilter),
    # transforms.Lambda(Notch),
    transforms.BaselineRemoval(),
    transforms.MeanStdNormalize(),
    transforms.To2d()
    
])

In [8]:
# 3. Load Data
dataset = SEEDIVDataset(
    io_path='./tmp_out/seed_iv',
    root_path='/kaggle/input/seed-iv/eeg_raw_data',
    offline_transform=t_transform,
    label_transform=transforms.Compose([
        transforms.Select('emotion'),
    ]),
    chunk_size=800,  # 4 seconds
    num_worker=4
)

[2025-12-01 20:37:58] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./tmp_out/seed_iv.
[2025-12-01 20:37:58] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/45 [00:00<?, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 0it [00:00, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 1it [00:04,  4.86s/it]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 8it [00:04,  2.19it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 15it [00:05,  4.82it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 22it [00:05,  8.23it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 30it [00:05, 13.17it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 38it [00:05, 19.00it/s]
[RECORD /kaggle/input/s

In [9]:
import pandas as pd

# 1. Get the metadata DataFrame
df = dataset.info

# 2. Count the segments for each emotion
# 0: Neutral, 1: Sad, 2: Fear, 3: Happy
counts = df['emotion'].value_counts().sort_index()
total = len(df)

print(f"Total Segments: {total}")
print("-" * 30)
print("Count per Emotion:")
print(counts)

print("-" * 30)
print("Percentage per Emotion:")
percentages = (counts / total) * 100
print(percentages.round(2))

# 3. Check for Imbalance
# If the difference between max and min is > 10%, we might need a WeightedSampler
max_pct = percentages.max()
min_pct = percentages.min()

if (max_pct - min_pct) > 10:
    print(f"\n⚠️ WARNING: Data is IMBALANCED (Diff: {max_pct - min_pct:.2f}%)")
    print("Consider using a WeightedRandomSampler.")
else:
    print(f"\n✅ Data is reasonably BALANCED (Diff: {max_pct - min_pct:.2f}%)")

Total Segments: 37575
------------------------------
Count per Emotion:
emotion
0    10170
1    10245
2     9225
3     7935
Name: count, dtype: int64
------------------------------
Percentage per Emotion:
emotion
0    27.07
1    27.27
2    24.55
3    21.12
Name: count, dtype: float64

✅ Data is reasonably BALANCED (Diff: 6.15%)


In [10]:
# Split by Trial ID
# SEED-IV has 24 trials (videos) per session.
# 80% of VIDEOS for training (19 videos), 20% for testing (5 videos).
all_trial_ids = list(range(1, 25))

random.seed(42)
test_trial_ids = random.sample(all_trial_ids, 5)
train_trial_ids = [t for t in all_trial_ids if t not in test_trial_ids]

train_indices = df[df['trial_id'].isin(train_trial_ids)].index.tolist()
test_indices = df[df['trial_id'].isin(test_trial_ids)].index.tolist()

# Create Subsets & Loaders
train_set = Subset(dataset, train_indices)
test_set = Subset(dataset, test_indices)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

In [11]:
import warnings
from collections import namedtuple
from typing import Any, Callable, Optional

import torch
import torch.nn as nn
from torch import Tensor
from torchvision.models.googlenet import BasicConv2d, Inception, InceptionAux
# Assuming BasicConv2d, Inception, InceptionAux are defined elsewhere, 
# as in torchvision, or included in your actual setup.
# For this code to run, you would need to define or import them.

# --- Definitions (Assuming the originals are used, but with reduced params) ---
# We keep the original namedtuple structure for type hints
GoogLeNetOutputs = namedtuple(
    "GoogLeNetOutputs", ["logits", "aux_logits2", "aux_logits1"]
)
GoogLeNetOutputs.__annotations__ = {
    "logits": Tensor,
    "aux_logits2": Optional[Tensor],
    "aux_logits1": Optional[Tensor],
}
_GoogLeNetOutputs = GoogLeNetOutputs


# Placeholder for original blocks if you don't have them imported:
# class BasicConv2d(nn.Module): ...
# class Inception(nn.Module): ...
# class InceptionAux(nn.Module): ...
# ----------------------------------------------------------------------------


class GoogLeNetLighter(nn.Module):
    __constants__ = ["aux_logits", "transform_input"]

    def __init__(
        self,
        num_classes: int = 4,
        aux_logits: bool = False, # Set to False for lighter model
        transform_input: bool = False,
        init_weights: Optional[bool] = None,
        blocks: Optional[list[Callable[..., nn.Module]]] = None,
        dropout: float = 0.4, # Increased dropout slightly
        dropout_aux: float = 0.7,
    ) -> None:
        super().__init__()
        if blocks is None:
            # Note: We must use the original blocks (BasicConv2d, Inception, InceptionAux)
            # but with reduced channel counts in the calls below.
            # Replace with your actual blocks if they are locally defined.
            conv_block = BasicConv2d
            inception_block = Inception
            inception_aux_block = InceptionAux
        else:
            if len(blocks) != 3:
                raise ValueError(f"blocks length should be 3 instead of {len(blocks)}")
            conv_block = blocks[0]
            inception_block = blocks[1]
            inception_aux_block = blocks[2]

        if init_weights is None:
            init_weights = True

        self.aux_logits = aux_logits
        self.transform_input = transform_input

        # 1. INPUT CHANNEL: 1 instead of 3 (62x800 input)
        # 2. REDUCED FILTERS & STRIDE: 
        #    - Filters: 64 -> 32
        #    - Kernel: 7x7 -> 5x5
        #    - Stride: 2 -> 1 (to preserve height)
        self.conv1 = conv_block(1, 32, kernel_size=5, stride=1, padding=2) # Output: N x 32 x 62 x 800
        
        # MaxPool with a larger stride on the width (800)
        self.maxpool1 = nn.MaxPool2d((3, 5), stride=(1, 2), ceil_mode=True) # Output: N x 32 x 62 x 400

        # REDUCED FILTERS: 64 -> 32
        self.conv2 = conv_block(32, 32, kernel_size=1) # Output: N x 32 x 62 x 400
        # REDUCED FILTERS: 192 -> 96
        self.conv3 = conv_block(32, 96, kernel_size=3, padding=1) # Output: N x 96 x 62 x 400
        
        # MaxPool with stride 2 on both dimensions to start downsampling
        self.maxpool2 = nn.MaxPool2d(3, stride=2, ceil_mode=True) # Output: N x 96 x 31 x 200

        # --- Inception Modules (Channels Halved/Reduced) ---
        # Original: (192, 64, 96, 128, 16, 32, 32) -> Output: 256
        # Reduced: (96, 32, 48, 64, 8, 16, 16) -> Output: 128
        self.inception3a = inception_block(96, 32, 48, 64, 8, 16, 16) # Output: N x 128 x 31 x 200
        # Original: (256, 128, 128, 192, 32, 96, 64) -> Output: 480
        # Reduced: (128, 64, 64, 96, 16, 48, 32) -> Output: 240
        self.inception3b = inception_block(128, 64, 64, 96, 16, 48, 32) # Output: N x 240 x 31 x 200
        
        self.maxpool3 = nn.MaxPool2d(3, stride=2, ceil_mode=True) # Output: N x 240 x 16 x 100

        # Original: (480, 192, 96, 208, 16, 48, 64) -> Output: 512
        # Reduced: (240, 96, 48, 104, 8, 24, 32) -> Output: 256
        self.inception4a = inception_block(240, 96, 48, 104, 8, 24, 32) # Output: N x 256 x 16 x 100
        
        # AUXILIARY HEADS REMOVED
        if aux_logits:
            warnings.warn("Auxiliary heads are not recommended for the lighter model.")
            # Still defining them if aux_logits is forced to True
            self.aux1 = inception_aux_block(512, num_classes, dropout=dropout_aux)
            self.aux2 = inception_aux_block(528, num_classes, dropout=dropout_aux)
        else:
            self.aux1 = None  # type: ignore[assignment]
            self.aux2 = None  # type: ignore[assignment]

        # Original: (512, 160, 112, 224, 24, 64, 64) -> Output: 512
        # Reduced: (256, 80, 56, 112, 12, 32, 32) -> Output: 256
        self.inception4b = inception_block(256, 80, 56, 112, 12, 32, 32) # Output: N x 256 x 16 x 100
        # Original: (512, 128, 128, 256, 24, 64, 64) -> Output: 512
        # Reduced: (256, 64, 64, 128, 12, 32, 32) -> Output: 256
        self.inception4c = inception_block(256, 64, 64, 128, 12, 32, 32) # Output: N x 256 x 16 x 100
        # Original: (512, 112, 144, 288, 32, 64, 64) -> Output: 528
        # Reduced: (256, 56, 72, 144, 16, 32, 32) -> Output: 264
        # self.inception4d = inception_block(256, 56, 72, 144, 16, 32, 32) # Output: N x 264 x 16 x 100
        # Original: (528, 256, 160, 320, 32, 128, 128) -> Output: 832
        # Reduced: (264, 128, 80, 160, 16, 64, 64) -> Output: 416
        # self.inception4e = inception_block(264, 128, 80, 160, 16, 64, 64) # Output: N x 416 x 16 x 100
        
        self.maxpool4 = nn.MaxPool2d((2, 4), stride=(2, 2), ceil_mode=True) # Output: N x 416 x 8 x 50

        # Original: (832, 256, 160, 320, 32, 128, 128) -> Output: 832
        # Reduced: (416, 128, 80, 160, 16, 64, 64) -> Output: 416
        self.inception5a = inception_block(256, 128, 80, 160, 16, 64, 64) # Output: N x 416 x 8 x 50
        # Original: (832, 384, 192, 384, 48, 128, 128) -> Output: 1024
        # Reduced: (416, 192, 96, 192, 24, 64, 64) -> Output: 512
        self.inception5b = inception_block(416, 192, 96, 192, 24, 64, 64) # Output: N x 512 x 8 x 50

        # Changed to reduce the output size of the inception block, 
        # so final feature map depth is 512, not 1024.
        
        # Global Average Pooling still pools down to (1, 1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) # Output: N x 512 x 1 x 1
        self.dropout = nn.Dropout(p=dropout)
        
        # Reduced input to Linear layer: 1024 -> 512
        self.fc = nn.Linear(512, num_classes)

        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                    torch.nn.init.trunc_normal_(m.weight, mean=0.0, std=0.01, a=-2, b=2)
                elif isinstance(m, nn.BatchNorm2d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)

    # _transform_input is only relevant for 3-channel image data, keeping it for completeness 
    # but it won't be used with 1-channel data.
    def _transform_input(self, x: Tensor) -> Tensor:
        if self.transform_input:
            warnings.warn("Input transformation is for 3-channel data and may not be correct for 1-channel signal.")
            # Original code assumes 3 channels, needs adaptation for 1 channel if normalization is desired
        return x

    def _forward(self, x: Tensor) -> tuple[Tensor, Optional[Tensor], Optional[Tensor]]:
        # N x 1 x 62 x 800
        x = self.conv1(x) # N x 32 x 62 x 800
        x = self.maxpool1(x) # N x 32 x 62 x 400
        x = self.conv2(x) # N x 32 x 62 x 400
        x = self.conv3(x) # N x 96 x 62 x 400
        x = self.maxpool2(x) # N x 96 x 31 x 200

        x = self.inception3a(x) # N x 128 x 31 x 200
        x = self.inception3b(x) # N x 240 x 31 x 200
        x = self.maxpool3(x) # N x 240 x 16 x 100

        x = self.inception4a(x) # N x 256 x 16 x 100
        
        # AUXILIARY HEADS REMOVED
        aux1: Optional[Tensor] = None
        aux2: Optional[Tensor] = None
        # if self.aux1 is not None: ...
        x = self.inception4b(x) # N x 256 x 16 x 100
        x = self.inception4c(x) # N x 256 x 16 x 100
        # x = self.inception4d(x) # N x 264 x 16 x 100
        
        # if self.aux2 is not None: ...

        # x = self.inception4e(x) # N x 416 x 16 x 100
        x = self.maxpool4(x) # N x 416 x 8 x 50
        x = self.inception5a(x) # N x 416 x 8 x 50
        x = self.inception5b(x) # N x 512 x 8 x 50

        x = self.avgpool(x) # N x 512 x 1 x 1
        x = torch.flatten(x, 1) # N x 512
        x = self.dropout(x)
        x = self.fc(x) # N x num_classes
        
        return x, aux2, aux1

    @torch.jit.unused
    def eager_outputs(
        self, x: Tensor, aux2: Optional[Tensor], aux1: Optional[Tensor]
    ) -> GoogLeNetOutputs:
        if self.training and self.aux_logits:
            return _GoogLeNetOutputs(x, aux2, aux1)
        else:
            return x  # type: ignore[return-value]

    def forward(self, x: Tensor) -> GoogLeNetOutputs:
        x = self._transform_input(x)
        x, aux2, aux1 = self._forward(x)
        aux_defined = self.training and self.aux_logits
        if torch.jit.is_scripting():
            if not aux_defined:
                warnings.warn(
                    "Scripted GoogleNet always returns GoogleNetOutputs Tuple"
                )
            return GoogLeNetOutputs(x, aux2, aux1)
        else:
            # We explicitly pass None for aux2, aux1 since aux_logits=False
            return self.eager_outputs(x, None, None) 


# The googlenet factory function would also need to be updated 
# to use GoogLeNetLighter and potentially remove the weights loading 
# since a custom architecture is used.

In [12]:
model = GoogLeNetLighter(init_weights=True, dropout=0.9).to(device)

In [13]:
# 5. Define Model
# model = EEGNet(
#     chunk_size=800,
#     num_electrodes=62,
#     dropout=0.5,
#     kernel_1=64,
#     kernel_2=16,
#     F1=8,
#     F2=16,
#     D=2,
#     num_classes=4  
# ).to(device)

In [14]:
# 6. Training Loop
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

In [15]:
patience = 15
counter = 0
best_val_acc = 0.0
best_model_state = None

for epoch in range(100):
    model.train()
    train_loss = 0
    correct_train = 0
    total_train = 0
    
    for batch in train_loader:
        X = batch[0].to(device).float()
        y = batch[1].to(device).long()
        
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += y.size(0)
        correct_train += (predicted == y).sum().item()
        
    avg_train_loss = train_loss / len(train_loader)
    train_acc = (correct_train / total_train) * 100

    # ==========================
    # 2. VALIDATION PHASE
    # ==========================
    model.eval() # Turn off dropout for accurate testing
    val_loss = 0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad(): # Don't calculate gradients for validation (saves memory)
        for batch in test_loader:
            X = batch[0].to(device).float()
            y = batch[1].to(device).long()
            
            outputs = model(X)
            loss = criterion(outputs, y)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += y.size(0)
            correct_val += (predicted == y).sum().item()
            
    avg_val_loss = val_loss / len(test_loader)
    val_acc = (correct_val / total_val) * 100

    print(f"Epoch {epoch+1}: Train Loss={avg_train_loss:.4f} (Acc={train_acc:.2f}%) | Val Loss={avg_val_loss:.4f} (Acc={val_acc:.2f}%)")
     # --- EARLY STOPPING ---
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), 'best_googlenet15_final.pth')
        print(f"  --> New Best! {best_val_acc:.2f}%")
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("  --> Early Stopping.")
            break

if best_model_state:
    model.load_state_dict(best_model_state)
    print(f"Finished. Best Acc: {best_val_acc:.2f}%")


Epoch 1: Train Loss=1.3384 (Acc=34.01%) | Val Loss=2.6543 (Acc=13.80%)
  --> New Best! 13.80%
Epoch 2: Train Loss=1.2233 (Acc=44.08%) | Val Loss=1.5793 (Acc=28.32%)
  --> New Best! 28.32%
Epoch 3: Train Loss=1.1209 (Acc=51.30%) | Val Loss=1.3616 (Acc=42.59%)
  --> New Best! 42.59%
Epoch 4: Train Loss=1.0295 (Acc=56.72%) | Val Loss=1.4436 (Acc=41.26%)
Epoch 5: Train Loss=0.9479 (Acc=61.35%) | Val Loss=1.3369 (Acc=42.99%)
  --> New Best! 42.99%
Epoch 6: Train Loss=0.8702 (Acc=65.37%) | Val Loss=1.3518 (Acc=46.34%)
  --> New Best! 46.34%
Epoch 7: Train Loss=0.7969 (Acc=68.66%) | Val Loss=1.3539 (Acc=47.49%)
  --> New Best! 47.49%
Epoch 8: Train Loss=0.7307 (Acc=71.91%) | Val Loss=1.5149 (Acc=41.94%)
Epoch 9: Train Loss=0.6768 (Acc=73.88%) | Val Loss=1.2840 (Acc=51.48%)
  --> New Best! 51.48%
Epoch 10: Train Loss=0.6213 (Acc=76.39%) | Val Loss=1.3318 (Acc=53.52%)
  --> New Best! 53.52%
Epoch 11: Train Loss=0.5722 (Acc=78.45%) | Val Loss=1.4487 (Acc=50.13%)
Epoch 12: Train Loss=0.5393 (Acc=